In [233]:
import numpy as np
import matplotlib.pyplot as plt

In [234]:
from loaders import clean_text_for_pattern_letter
from temp_text_puh import template_str
# input_str = clean_text_for_pattern_letter(template_str, out_len=len(template_str))
input_str = clean_text_for_pattern_letter(template_str, out_len=5000)
print(len(input_str))
letters = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u',
               'v', 'w', 'x', 'y', 'z', ' ', '.', ',', '"', '\'', '1']
print(f'{letters=}')

len(letters)=32
5000
letters=['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', ' ', '.', ',', '"', "'", '1']


In [235]:
def predict_train(input_str,
                  letters,
                  max_history=4,
                  max_index: int = 4,):
    WIN: dict[str, dict[str, int]] = {}

    HistoryHW: list[dict[str, int]] = []
    HistoryW: list[dict[str, dict[str, dict[int, int]]]] = []

    for h_idx in range(max_history):
        HistoryHW.append({})
        HistoryW.append({})

    IDX: list = []

    for letter in letters:
        WIN.setdefault(letter, {})

        for idx in range(max_index):
            WIN[letter].setdefault(idx, 0)

    prev_indices = [0]

    for inp_idx, inp in enumerate(input_str):
        if inp_idx <= 0:
            WIN[inp][prev_indices[0]] += 1
            continue

        IDX.append(prev_indices[0])

        IdxO = {}

        for h_idx in range(max_history):
            prev_idx = inp_idx - 1 - h_idx
            if prev_idx < 0:
                break

            prev_inp = input_str[prev_idx]

            HW = HistoryHW[h_idx]
            W = HistoryW[h_idx]

            key_HW = prev_inp + str(prev_indices[h_idx]) + ':' + inp

            if HW.get(key_HW, -1) != -1:
                inp_has_index = HW[key_HW]

                IdxO.setdefault(inp_has_index, 0)
                IdxO[inp_has_index] += 1

        new_index = 0
        if len(IdxO) == 0:
            new_index = min(WIN[inp], key=WIN[inp].get)
            WIN[inp][new_index] += 1
        else:
            new_index = max(IdxO, key=IdxO.get)

        for h_idx in range(max_history):
            prev_idx = inp_idx - 1 - h_idx
            if prev_idx < 0:
                break

            prev_inp = input_str[prev_idx]

            key_I = prev_inp + str(prev_indices[h_idx])
            HistoryW[h_idx].setdefault(key_I, {})
            HistoryW[h_idx][key_I].setdefault(inp, {})
            HistoryW[h_idx][key_I][inp].setdefault(new_index, 0)
            HistoryW[h_idx][key_I][inp][new_index] += 1

            key_HW = prev_inp + str(prev_indices[h_idx]) + ':' + inp

            HistoryHW[h_idx].setdefault(key_HW, new_index)

        # recurtion
        prev_indices.insert(0, new_index)
        if len(prev_indices) > max_history:
            prev_indices = prev_indices[:-1]

    return HistoryW, HistoryHW, IDX


max_index_L1 = 32
max_history_L1 = 1

DSHistoryW, HistoryHW, IDX = predict_train(input_str, letters, max_history=max_history_L1, max_index=max_index_L1)

for h_idx in range(max_history_L1):
    for wk, wv in DSHistoryW[h_idx].items():
        print(wk, wv)


c0 {'h': {0: 1}, 'e': {4: 1}}
h0 {'a': {0: 1}, 't': {4: 1}, 'e': {11: 1}, ' ': {17: 2}}
a0 {'p': {0: 1}, 'n': {2: 1}, 't': {13: 2}, 'k': {16: 1}, 's': {23: 1}, 'd': {0: 1}}
p0 {'t': {0: 1}, 'l': {6: 1}}
t0 {'e': {0: 1}, 'o': {31: 2}, 'l': {18: 1}, 'h': {28: 8}, 'r': {28: 1}, ' ': {19: 2}, 'w': {8: 1}}
e0 {'r': {0: 2}, 'n': {20: 2}, 's': {7: 2}, 'd': {1: 1}, ' ': {17: 3}, 't': {2: 1}}
r0 {' ': {0: 4}, 'e': {11: 1}, 's': {24: 1}}
 0 {'i': {0: 5}, 'b': {7: 6}, 'f': {5: 1}, 'h': {31: 8}, 'w': {15: 1}, 'g': {10: 1}, 'a': {21: 7}, 'l': {17: 1}, 'o': {27: 2}, 't': {22: 2}, 'y': {28: 1}}
i0 {'n': {0: 6}, 's': {17: 1}, ' ': {11: 1}, 'k': {17: 1}, 't': {26: 2}}
n0 {' ': {1: 3}, 'a': {25: 1}, 'g': {5: 2}, 'd': {5: 2}, 'n': {15: 1}, 'e': {1: 2}, 'c': {3: 1}, 'k': {23: 1}}
 1 {'w': {0: 2}, 'c': {5: 3}, 'a': {18: 3}, 'n': {0: 1}, 'p': {12: 4}, 's': {9: 2}, 'i': {10: 4}, 't': {12: 5}, 'g': {14: 1}, 'b': {25: 3}, 'l': {23: 1}, 'h': {22: 1}, 'o': {3: 1}, 'm': {26: 1}, 'f': {15: 1}}
w0 {'h': {1: 1}, 'n'

In [236]:
def predict(input_str: str, 
            HistoryW: list[dict[str, dict[str, dict[int, int]]]], 
            max_history = 4,
            max_index = 4,):
    err = 0
    TF = []
    AL = []

    history_indices = []
    history_indices.append([x for x in range(max_index)])

    for input_idx, input in enumerate(input_str[:-1]):
        HO: dict[str, dict[int, int]] = {}

        for h_idx in range(max_history):
            inp_idx = input_idx - h_idx
            if inp_idx < 0:
                break
            # print(f'{h_idx=} {inp_idx=}')
            W = HistoryW[h_idx]
            inp = input_str[inp_idx]
            indices = history_indices[h_idx]

            key_I = inp + str(indices[0])
            O = W.get(key_I, {})
            # print(f'{O=}')
            # print()

            for index in indices[1:]:
                key_I = inp + str(index)

                index_O = W.get(key_I, {})
                # print(f'{index_O=}')
                # print()

                for key, val in index_O.items():
                    O.setdefault(key, {})
                    O[key].update(val)
            
            # print(f'{O=}')
            # print()
            
            for key_O, val_O in O.items():
                HO.setdefault(key_O, {})
                
                for key_index, val_index in val_O.items():
                    HO[key_O].setdefault(key_index, 0)

                    HO[key_O][key_index] += val_index

        # print(f'{HO=}')
        # print()

        length = 0
        for v in HO.values():
            length += len(v)

        max_arr = []
        for v in HO.values():
            col_max = max(v, key=v.get)
            max_arr.append(col_max)
        
        max_arr = np.array(max_arr)
        # tf = np.mean(np.abs(max_arr - max_arr[np.newaxis].T))

        tf = length
        # tf = len(HO)

        TF.append(tf)

        max_symbol = 'a'
        max_symbol_index = 0
        max_val = 0
        for key, val in HO.items():
            max_idx = max(val, key=val.get)
            HO[key] = {max_idx: val[max_idx]}

            if val[max_idx] > max_val:
                max_val = val[max_idx]
                max_symbol_index = max_idx
                max_symbol = key

        # print(f'{HO=}')
        next_inp = input_str[input_idx + 1]
        print(f'{max_symbol=}', f'{next_inp=}', max_symbol_index, max_val)

        # anomaly
        if max_symbol == next_inp:
            AL.append(0)
        else:
            ni_v = HO.get(next_inp, 0)
            if ni_v == 0:
                AL.append(100)
            else:
                max_ni = max(ni_v, key=ni_v.get)
                AL.append(abs(ni_v[max_ni] - max_val))

        new_indices = [max_symbol_index]

        # err
        if max_symbol != next_inp:
            err += 1

            index_stats = HO.get(next_inp, 0)
            
            if index_stats == 0:
                new_indices = [x for x in range(max_index)]
                print(f'{new_indices=}')
            else:
                max_symbol_index = list(index_stats.keys())[0]

                new_indices = [max_symbol_index]
        
        # recurtion
        history_indices.insert(0, new_indices)
        if len(history_indices) > max_history:
            history_indices = history_indices[:-1]

    TF.append(0)
    AL.append(0)
    return err, TF, AL

err_L1, TF_L1, AL_L1 = predict(input_str, DSHistoryW, max_index=max_index_L1, max_history=max_history_L1)

print(err_L1, err_L1 / (len(input_str) / 100))

max_symbol='h' next_inp='h' 17 3
max_symbol='e' next_inp='a' 27 3
max_symbol='t' next_inp='p' 15 2
new_indices=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31]
max_symbol='h' next_inp='t' 3 5
max_symbol='h' next_inp='e' 28 8
max_symbol=' ' next_inp='r' 17 3
max_symbol=' ' next_inp=' ' 0 4
max_symbol='h' next_inp='i' 31 8
max_symbol='n' next_inp='n' 0 6
max_symbol=' ' next_inp=' ' 1 3
max_symbol='t' next_inp='w' 12 5
max_symbol='h' next_inp='h' 1 1
max_symbol='i' next_inp='i' 1 5
max_symbol='n' next_inp='c' 20 5
max_symbol='h' next_inp='h' 2 2
max_symbol=' ' next_inp=' ' 2 3
max_symbol='w' next_inp='w' 1 6
max_symbol='e' next_inp='e' 1 4
max_symbol=' ' next_inp=' ' 3 11
max_symbol='a' next_inp='a' 1 8
max_symbol='n' next_inp='r' 21 6
max_symbol='e' next_inp='e' 2 2
max_symbol=' ' next_inp=' ' 4 14
max_symbol='s' next_inp='i' 14 6
max_symbol='d' next_inp='n' 19 5
max_symbol=' ' next_inp='t' 23 4
max_symbol=' ' next_inp

In [237]:
def get_word_stats(input_str: str | list, TF: list, threshold=0.0):
    buffer: list = [input_str[0]]
    WordStats: dict[str, int] = {}
    WordIndex: dict[str, int] = {}
    next_wrd_idx = 0

    NextLayerOut = []
    DownstreamIndex = {}

    prev_tf = TF[0]

    for inp_idx, inp in enumerate(input_str):
        if inp_idx <= 0 and inp_idx >= len(input_str) - 1:
            continue

        # if TF[inp_idx] - prev_tf > 0.0: # and abs(TF[inp_idx] - prev_tf) > 0.2:
        # and TF[inp_idx] > threshold:
        if TF[inp_idx] > prev_tf and TF[inp_idx] > TF[inp_idx + 1]:
            word = ''.join(buffer)

            WordStats.setdefault(word, 0)
            WordStats[word] += 1

            if not WordIndex.get(word):
                WordIndex.setdefault(word, next_wrd_idx)
                next_wrd_idx += 1

            wrd_idx = WordIndex.get(word)
            # NextLayerOut.append(str(wrd_idx))
            NextLayerOut.append(word)

            DownstreamIndex.setdefault(inp_idx, 1)

            buffer = [inp]
            prev_tf = TF[inp_idx]

            continue

        TF[inp_idx] = 0

        buffer.append(inp)
        prev_tf = TF[inp_idx]

    # sort
    WordStats = dict(sorted(WordStats.items(), key=lambda item: item[1], reverse=True))

    NextLayerOut.append(str(next_wrd_idx))
    return WordStats, WordIndex, NextLayerOut, DownstreamIndex


TF_L1 = np.array(TF_L1, dtype='float64')
AL_L1 = np.array(AL_L1, dtype='float64')
TF_L1 /= np.max(TF_L1)
AL_L1 /= np.max(AL_L1)
TF_L1 += AL_L1

TF_max_coef = 0.4

# TF_L1[np.where(TF_L1 < np.mean(TF_L1))] = 0
# TF_1 = TF_L1 - np.pad(TF_L1, (1, 0))[:-1]
# TF_2 = TF_L1 - np.pad(TF_L1, (0, 1))[1:]
# TF_L1 = TF_1 + TF_2

# TF_L1[np.where(TF_L1 < 0)] = 0
# TF_L1[np.where(TF_L1 < TF_max_coef)] = 0
TF_L1 = TF_L1.tolist()

WordStats_L1, WordIndex_L1, NextLayerOut_L1, DownstreamIndex_L1 = get_word_stats(input_str=input_str,
                                                                                 TF=TF_L1, 
                                                                                 threshold=0.0, )
print(len(WordStats_L1))
mean_WS = np.mean(np.array(list(WordStats_L1.values())))
print(f'{mean_WS=}')
WS_len = 0
for key, val in WordStats_L1.items():
    # if len(key.strip()) > 1 and val >= mean_WS:
    if val >= mean_WS:
        WS_len += 1
    print(key, val)
print(f'{WS_len=}')

print(f'{NextLayerOut_L1=}')
print(f'{len(NextLayerOut_L1)=}')

print(f'{WordIndex_L1=}')

print(f'{DownstreamIndex_L1=}')


512
mean_WS=3.814453125
 a 71
 b 54
 s 43
 he 40
 the 39
nd 38
 h 33
 i 33
er 31
 w 29
 t 28
ng 24
 y 22
ou 22
 c 21
 of 21
 r 21
ed 20
es 20
 l 20
 n 19
 th 19
aid 19
hat 18
 o 17
en 17
st 15
 f 14
 m 14
 g 14
 d 13
 to 12
ow 12
 it 12
om 12
 do 12
obin 11
as 11
is 10
ing 10
et 10
ea 10
 p 10
se 10
ie 9
 is 9
to 9
id 9
oph 9
all 9
ee 9
ns 8
ad 8
 as 8
re 8
 if 8
or 8
hi 8
 po 8
oi 8
 wi 8
at 8
 in 7
ar 7
oh 7
 sa 7
od 6
 be 6
nly 6
ng  6
 so 6
 v 6
elf 6
on 6
ot 6
uzz 6
ey 6
 cl 6
imb 6
 wh 5
tr 5
ome 5
 st 5
ay 5
 fe 5
ly 5
th 5
n't 5
ame 5
ht 5
nn 5
out 5
ri 5
ho 5
nch 5
ere 4
 bu 4
nt 4
nk 4
hr 4
t  4
 i  4
rs 4
 to  4
 chri 4
ery 4
ne 4
ims 4
 an 4
 the  4
uzzi 4
it 4
nd  4
ug 4
tle 4
 fu 4
ra 4
el 4
ty 4
 go 4
ut 4
ou  4
uc 3
 pooh 3
ard 3
irs 3
 on 3
 kn 3
im 3
 at 3
tt 3
ed  3
ir 3
 j 3
ust 3
e's 3
oo 3
ec 3
au 3
tim 3
nnie 3
ik 3
 fr 3
ory 3
 ab 3
ld 3
ab 3
 up 3
 lo 3
time 3
iv 3
 fo 3
 by 3
der 3
 wa 3
 gr 3
sa 3
ace 3
 of  3
eason 3
aki 3
 yo 3
oth 3
ei 3
ong 3
hon 3
 ju 3


In [238]:
def downstream_train(input: list, next_layer_out: list, downstream_index: dict[int, int]):
    DSStats = {}

    ds_idx = 1
    ds_inp = next_layer_out[0]

    for inp_idx, inp in enumerate(input[:-1]):
        has_ds_input = downstream_index.get(inp_idx)
        if has_ds_input:
            ds_inp = next_layer_out[ds_idx]
            ds_idx += 1

        ds_key = ds_inp + ':' + inp
        DSStats.setdefault(ds_key, 0)
        DSStats[ds_key] += 1

    return DSStats


# DSStats = downstream_train(input=input_str,
#                            next_layer_out=NextLayerOut_L1,
#                            downstream_index=DownstreamIndex_L1,)

# for wk, wv in DSStats.items():
#     print(wk, wv)

In [239]:
def downstream_predict_train(input_str: str,
                             upstream_inp: dict[int, list],
                             ds_input: list,
                             max_history=4,
                             max_index=4,):
    WIN: dict[str, dict[str, int]] = {}

    HistoryHW: list[dict[str, int]] = []
    HistoryW: list[dict[str, dict[str, dict[int, int]]]] = []

    for h_idx in range(max_history):
        HistoryHW.append({})
        HistoryW.append({})

    IDX: list = []

    for letter in letters:
        WIN.setdefault(letter, {})

        for idx in range(max_index):
            WIN[letter].setdefault(idx, 0)

    prev_indices = [0]

    ds_idx = 1
    ds_inp = ds_input[0]

    for inp_idx, inp in enumerate(input_str):
        if inp_idx <= 0:
            WIN[inp][prev_indices[0]] += 1
            continue

        IDX.append(prev_indices[0])

        IdxO = {}

        for h_idx in range(max_history):
            prev_idx = inp_idx - 1 - h_idx
            if prev_idx < 0:
                break

            prev_inp = input_str[prev_idx]

            HW = HistoryHW[h_idx]
            W = HistoryW[h_idx]

            key_HW = prev_inp + str(prev_indices[h_idx]) + ':' + inp

            if HW.get(key_HW, -1) != -1:
                inp_has_index = HW[key_HW]

                IdxO.setdefault(inp_has_index, 0)
                IdxO[inp_has_index] += 1

        new_index = 0
        if len(IdxO) == 0:
            new_index = min(WIN[inp], key=WIN[inp].get)
            WIN[inp][new_index] += 1
        else:
            new_index = max(IdxO, key=IdxO.get)

        # ds input
        has_ds_input = upstream_inp.get(inp_idx - 1)
        if has_ds_input:
            ds_inp = ds_input[ds_idx]
            ds_idx += 1

        for h_idx in range(max_history):
            prev_idx = inp_idx - 1 - h_idx
            if prev_idx < 0:
                break

            prev_inp = input_str[prev_idx]

            key_I = ds_inp + ':' + prev_inp + str(prev_indices[h_idx])
            HistoryW[h_idx].setdefault(key_I, {})
            HistoryW[h_idx][key_I].setdefault(inp, {})
            HistoryW[h_idx][key_I][inp].setdefault(new_index, 0)
            HistoryW[h_idx][key_I][inp][new_index] += 1

            key_HW = prev_inp + str(prev_indices[h_idx]) + ':' + inp

            HistoryHW[h_idx].setdefault(key_HW, new_index)

        # recurtion
        prev_indices.insert(0, new_index)
        if len(prev_indices) > max_history:
            prev_indices = prev_indices[:-1]

    return HistoryW, HistoryHW, IDX


HistoryW_L1_ds, HistoryHW_L1_ds, IDX_L1_ds = downstream_predict_train(input_str=input_str,
                                                                      upstream_inp=DownstreamIndex_L1,
                                                                      ds_input=NextLayerOut_L1,
                                                                      max_history=max_history_L1,
                                                                      max_index=max_index_L1,)

for h_idx in range(max_history_L1):
    for wk, wv in HistoryW_L1_ds[h_idx].items():
        print(wk, wv)


cch:c0 {'h': {0: 1}}
cch:h0 {'a': {0: 1}}
ap:a0 {'p': {0: 1}}
ap:p0 {'t': {0: 1}}
ter:t0 {'e': {0: 1}}
ter:e0 {'r': {0: 1}}
ter:r0 {' ': {0: 1}}
 in: 0 {'i': {0: 2}}
 in:i0 {'n': {0: 2}}
 in:n0 {' ': {1: 2}}
 wh: 1 {'w': {0: 1}}
 wh:w0 {'h': {1: 1}}
 wh:h1 {'i': {1: 1}}
ich:i1 {'c': {1: 1}}
ich:c1 {'h': {2: 1}}
ich:h2 {' ': {2: 1}}
 we : 2 {'w': {1: 1}}
 we :w1 {'e': {1: 1}}
 we :e1 {' ': {3: 1}}
 we : 3 {'a': {1: 1}}
are:a1 {'r': {1: 1}}
are:r1 {'e': {2: 1}}
are:e2 {' ': {4: 1}}
 in: 4 {'i': {2: 1}}
 in:i2 {'n': {1: 1}}
 in:n1 {'t': {1: 1}}
tr:t1 {'r': {2: 1}}
tr:r2 {'o': {0: 1}}
od:o0 {'d': {0: 1}}
od:d0 {'u': {0: 1}}
uc:u0 {'c': {2: 1}}
uc:c2 {'e': {3: 1}}
ed:e3 {'d': {1: 2}}
ed:d1 {' ': {5: 4}}
 to: 5 {'t': {2: 2}}
 to:t2 {'o': {1: 2}}
 to:o1 {' ': {6: 2}}
 w: 6 {'w': {2: 1}}
 w:w2 {'i': {3: 1}}
inn:i3 {'n': {2: 1}}
inn:n2 {'n': {3: 1}}
inn:n3 {'i': {4: 1}}
ie:i4 {'e': {4: 2}}
ie:e4 {' ': {7: 2}}
 the: 7 {'t': {3: 5}}
 the:t3 {'h': {3: 5}}
 the:h3 {'e': {5: 5}}
 the:e5 {' ': {8: 4}

In [240]:
def downstream_predict(input_str: str,
                       DSHistoryW: list[dict[str, dict[str, dict[int, int]]]],
                       upstream_inp: dict[int, list],
                       ds_input: list,
                       max_history=4,
                       max_index=4,):
    err = 0
    TF = []

    history_indices = []
    history_indices.append([x for x in range(max_index)])

    ds_idx = 1
    ds_inp = ds_input[0]

    for input_idx, input in enumerate(input_str[:-1]):
        HO: dict[str, dict[int, int]] = {}

        # ds input
        has_ds_input = upstream_inp.get(input_idx)
        if has_ds_input:
            ds_inp = ds_input[ds_idx]
            ds_idx += 1

        for h_idx in range(max_history):
            inp_idx = input_idx - h_idx
            if inp_idx < 0:
                break
            # print(f'{h_idx=} {inp_idx=}')
            W = DSHistoryW[h_idx]
            inp = input_str[inp_idx]
            indices = history_indices[h_idx]

            key_I = ds_inp + ':' + inp + str(indices[0])
            # print(f'{key_I=}')
            O = W.get(key_I, {})
            # print(f'{O=}')
            # print()

            for index in indices[1:]:
                key_I = ds_inp + ':' + inp + str(index)
                # print(f'{key_I=}')

                index_O = W.get(key_I, {})
                # print(f'{index_O=}')
                # print()

                for key, val in index_O.items():
                    O.setdefault(key, {})
                    O[key].update(val)

            # print(f'{O=}')
            # print()

            for key_O, val_O in O.items():
                HO.setdefault(key_O, {})

                for key_index, val_index in val_O.items():
                    HO[key_O].setdefault(key_index, 0)

                    HO[key_O][key_index] += val_index

        # print(f'{HO=}')
        # print()

        length = 0
        for v in HO.values():
            length += len(v)

        # tf = length
        tf = len(HO)

        TF.append(tf)

        # max symbol
        max_symbol = 'a'
        max_symbol_index = 0
        max_val = 0
        for key, val in HO.items():
            max_idx = max(val, key=val.get)
            HO[key] = {max_idx: val[max_idx]}
            max_idx_val = val[max_idx]

            # ds predict
            # ds_key = ds_inp + ':' + key
            # ds_stats = DSStats.get(ds_key)
            # if ds_stats:
            #     max_idx_val += ds_stats

            if max_idx_val > max_val: # and ds_stats:
                max_val = max_idx_val
                max_symbol_index = max_idx
                max_symbol = key

        # print(f'{HO=}')
        next_inp = input_str[input_idx + 1]
        print(f'{max_symbol=}', f'{next_inp=} {ds_inp=} {HO=}',
              max_symbol_index, max_val)

        # print(f'{O.get(max_symbol).keys()=}')
        new_indices = [max_symbol_index]

        if O.get(max_symbol):
            new_indices = list(O.get(max_symbol).keys())

        # err
        if max_symbol != next_inp:
            err += 1

            index_stats = HO.get(next_inp, 0)

            if index_stats == 0:
                new_indices = [x for x in range(max_index)]
                print(f'{new_indices=}')
            else:
                max_symbol_index = list(index_stats.keys())[0]

                new_indices = [max_symbol_index]

        # recurtion
        history_indices.insert(0, new_indices)
        if len(history_indices) > max_history:
            history_indices = history_indices[:-1]

    return err, TF


err_L1_ds, TF_L1_ds = downstream_predict(input_str=input_str,
                                         DSHistoryW=HistoryW_L1_ds,
                                         upstream_inp=DownstreamIndex_L1,
                                         ds_input=NextLayerOut_L1,
                                         max_history=max_history_L1,
                                         max_index=max_index_L1, )

print(f'{err_L1=}', err_L1 / (len(input_str) / 100))
print(f'{err_L1_ds=}', err_L1_ds / (len(input_str) / 100))


max_symbol='h' next_inp='h' ds_inp='cch' HO={'h': {0: 1}} 0 1
max_symbol='a' next_inp='a' ds_inp='cch' HO={'a': {0: 1}} 0 1
max_symbol='p' next_inp='p' ds_inp='ap' HO={'p': {0: 1}} 0 1
max_symbol='t' next_inp='t' ds_inp='ap' HO={'t': {0: 1}} 0 1
max_symbol='e' next_inp='e' ds_inp='ter' HO={'e': {0: 1}} 0 1
max_symbol='r' next_inp='r' ds_inp='ter' HO={'r': {0: 1}} 0 1
max_symbol=' ' next_inp=' ' ds_inp='ter' HO={' ': {0: 1}} 0 1
max_symbol='i' next_inp='i' ds_inp=' in' HO={'i': {0: 2}} 0 2
max_symbol='n' next_inp='n' ds_inp=' in' HO={'n': {0: 2}} 0 2
max_symbol=' ' next_inp=' ' ds_inp=' in' HO={' ': {1: 2}} 1 2
max_symbol='w' next_inp='w' ds_inp=' wh' HO={'w': {0: 1}} 0 1
max_symbol='h' next_inp='h' ds_inp=' wh' HO={'h': {1: 1}} 1 1
max_symbol='i' next_inp='i' ds_inp=' wh' HO={'i': {1: 1}} 1 1
max_symbol='c' next_inp='c' ds_inp='ich' HO={'c': {1: 1}} 1 1
max_symbol='h' next_inp='h' ds_inp='ich' HO={'h': {2: 1}} 2 1
max_symbol=' ' next_inp=' ' ds_inp='ich' HO={' ': {2: 1}} 2 1
max_symbol